In [41]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor

In [4]:
cars_df = pd.read_csv('C:\PYTHON\Predicting_cost_car\Files/final_car_data_2023-12-17.csv')
print(cars_df.shape)
cars_df.head()

(15615, 13)


<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\фвьшт\AppData\Local\Temp\ipykernel_8508\3374424542.py:1: SyntaxWarning: invalid escape sequence '\P'
  cars_df = pd.read_csv('C:\PYTHON\Predicting_cost_car\Files/final_car_data_2023-12-17.csv')


,Brand,Model,Price,Year,Condition,Transmission,Mileage,Body_type,Drive_type,Count_horsepower,Engine_type,Usage_intensity,Origin
0,Audi,A6,1700000,2012,0,AMT,209000,седан,полный,245,дизель,17416.666667,European
1,Audi,Q7,2150000,2009,0,AT,5,внедорожник,полный,239,дизель,0.333333,European
2,Audi,A6,2700000,2015,0,AMT,149000,седан,полный,333,бензин,16555.555556,European
3,Audi,A5,5890000,2022,0,AMT,5400,лифтбек,полный,249,бензин,2700.000000,European
4,Audi,Q5,3150000,2019,0,AMT,61000,внедорожник,полный,163,дизель,12200.000000,European


In [48]:
y = cars_df["Price"]
X = cars_df.drop(columns=["Price"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=77)

## 1. Гипотезы 

## 2. Маштабирование и кодирование признаков

In [15]:
numeric_features = ['Year', 'Condition', 'Mileage', 'Count_horsepower', 'Usage_intensity']

categorical_features = ['Brand', 'Model', 'Transmission', 'Body_type', 'Drive_type', 'Engine_type', 'Origin']

In [16]:
column_transformer = ColumnTransformer([
    ('scaling', StandardScaler(), numeric_features),    
    ('ohe', OneHotEncoder(handle_unknown="ignore", drop="first"), categorical_features)
])

In [17]:
column_transformer

ColumnTransformer(transformers=[('scaling', StandardScaler(),
                                 ['Year', 'Condition', 'Mileage',
                                  'Count_horsepower', 'Usage_intensity']),
                                ('ohe',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['Brand', 'Model', 'Transmission', 'Body_type',
                                  'Drive_type', 'Engine_type', 'Origin'])])

In [20]:
transformer_names = column_transformer.get_feature_names_out()
transformer_names

array(['scaling__Year', 'scaling__Condition', 'scaling__Mileage',
       'scaling__Count_horsepower', 'scaling__Usage_intensity',
       'ohe__Brand_Audi', 'ohe__Brand_BMW', 'ohe__Brand_Chery',
       'ohe__Brand_Chevrolet', 'ohe__Brand_Daewoo', 'ohe__Brand_Datsun',
       'ohe__Brand_Ford', 'ohe__Brand_Geely', 'ohe__Brand_Haval',
       'ohe__Brand_Honda', 'ohe__Brand_Hyundai', 'ohe__Brand_Kia',
       'ohe__Brand_Lexus', 'ohe__Brand_Mazda', 'ohe__Brand_Mercedes-Benz',
       'ohe__Brand_Mitsubishi', 'ohe__Brand_Nissan', 'ohe__Brand_Opel',
       'ohe__Brand_Peugeot', 'ohe__Brand_Renault', 'ohe__Brand_Skoda',
       'ohe__Brand_Subaru', 'ohe__Brand_Suzuki', 'ohe__Brand_Toyota',
       'ohe__Brand_Volkswagen', 'ohe__Brand_ВАЗ', 'ohe__Brand_ГАЗ',
       'ohe__Brand_УАЗ', 'ohe__Model_(LADA) 2101',
       'ohe__Model_(LADA) 2102', 'ohe__Model_(LADA) 2103',
       'ohe__Model_(LADA) 2104', 'ohe__Model_(LADA) 2105',
       'ohe__Model_(LADA) 2106', 'ohe__Model_(LADA) 2107',
       'ohe__Mod

In [18]:
X_train_scaled = pd.DataFrame(column_transformer.fit_transform(X_train).toarray())
X_test_scaled = pd.DataFrame(column_transformer.transform(X_test).toarray())

c:\Users\фвьшт\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## 3. Обучение Линейной Регрессии

Линейная регрессия может быть полезна в такой задаче из-за своей простоты и интуитивной интерпретируемости. Она позволяет оценить линейную зависимость между целевой переменной и набором признаков. Кроме того, линейная регрессия хорошо работает с большим количеством признаков и данными, что делает ее подходящим выбором для данной задачи

In [74]:
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

y_train_pred = lr_model.predict(X_train_scaled)
y_pred = lr_model.predict(X_test_scaled)

In [75]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 469773127420.82654
Test MSE = 636996932771.5793

Train MAE = 337222.56179162924
Test MAE = 374949.84728024824


In [76]:
# Создадим таблицу с весами
lr_base_feature_coefs = pd.DataFrame(
    data=lr_model.coef_, index=transformer_names, columns=['Coef']
)

lr_base_feature_coefs.sort_values(by='Coef')

,Coef
ohe__Model_M-класс AMG,-4.804956e+06
ohe__Model_X6 M,-3.696233e+06
ohe__Model_M-класс,-2.579748e+06
ohe__Model_B-класс,-2.512249e+06
ohe__Model_GL-класс AMG,-2.445120e+06
...,...
ohe__Model_G-класс AMG,8.378565e+06
ohe__Model_Sequoia,8.584796e+06
ohe__Model_Kapitan,8.845538e+06
ohe__Model_Corvette,1.053387e+07


__Вывод:__ Итак, посмотрев на получившиеся оптимальные веса модели, нетрудно заметить симптом переобучения, подгонки нашей модели под данные, - большие веса у признаков. Более того, обобщающая способность модели получается низкой, ведь она оказывается не способна выдавать хорошие резльтаты как на тренировочной выборке, так и на тестовой.

## 3) Decision Tree

Случайный лес и решающее дерево являются более сложными моделями, которые могут улавливать нелинейные зависимости между признаками и целевой переменной. Эти модели могут быть эффективны в предсказании цены машин, особенно если данные содержат сложные взаимосвязи между признаками. Кроме того, случайный лес и решающее дерево способны обрабатывать большое количество признаков без потери производительности

In [80]:
dt_model = DecisionTreeRegressor()

Задаем сетку параметров для поиска

In [97]:
dt_param_grid = {
    'max_depth': [5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 5, 10]
}

Использование `RandomizedSearchCV` вместо `GridSearchCV`  на больших объемах данных позволяет проводить более эффективный и быстрый поиск оптимальных гиперпараметров для моделей машинного обучения.

In [98]:
dt_grid_search = RandomizedSearchCV(dt_model, dt_param_grid, cv=5)
dt_grid_search.fit(X_train_scaled, y_train)

RandomizedSearchCV(cv=5,
                   estimator=DecisionTreeRegressor(max_depth=25,
                                                   min_samples_split=5),
                   param_distributions={'max_depth': [5, 10, 15, 20, 25],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15]})

In [99]:
# Выводим лучшие параметры модели
print("Лучшие параметры модели:", dt_grid_search.best_params_)

Лучшие параметры модели: {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 25}


In [100]:
dt_best_min_samples_split, dt_best_min_samples_leaf, dt_best_max_depth = dt_grid_search.best_params_.values()
dt_best_min_samples_split, dt_best_min_samples_leaf, dt_best_max_depth 

(10, 1, 25)

In [101]:
dt_model = DecisionTreeRegressor(
    min_samples_split=dt_best_min_samples_split, min_samples_leaf=dt_best_min_samples_leaf ,max_depth=dt_best_max_depth
)

dt_model.fit(X_train_scaled, y_train)

DecisionTreeRegressor(max_depth=25, min_samples_split=10)

In [102]:
y_train_pred = dt_model.predict(X_train_scaled)
y_pred = dt_model.predict(X_test_scaled)

In [103]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 87063895219.45853
Test MSE = 299190553116.67175

Train MAE = 125828.99062199492
Test MAE = 261123.8987171451


In [104]:
# Создадим таблицу с важностями
dt_model_feature_coefs = pd.DataFrame(
    data=dt_model.feature_importances_, index=transformer_names, columns=['Coef']
)

dt_model_feature_coefs.sort_values(by='Coef', ascending=False)

,Coef
scaling__Count_horsepower,0.612903
scaling__Year,0.188853
scaling__Usage_intensity,0.034091
scaling__Mileage,0.031301
ohe__Brand_Mercedes-Benz,0.027606
...,...
ohe__Model_Mark II Wagon Qualis,0.000000
ohe__Model_Mark II,0.000000
ohe__Model_Malibu,0.000000
ohe__Model_MPV,0.000000


## 4) Random Forest

Теперь обучим нашу модель с применение случайного леса. В машинном обучении случайный лес считается довольно распространенный алгоритмом, который помогает улучшать качество построенных моделей за счет независимого обучения большого числа деревьев. Более того, данный алгоритм не склонен к переобучению, ведь даже при увеличении числа деревьев ошибка выходит на асимптоту.

Применими модель Random Forest и произведем перебор по сетке с целью нахождения оптимальный гиперпараметров.

In [108]:
rf_model = RandomForestRegressor()

Зададим сетку параметров для поиска

In [118]:
rf_param_grid = {
    'n_estimators': [100],
    'max_depth': [3, 5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

In [119]:
rf_grid_search = RandomizedSearchCV(rf_model, rf_param_grid, cv=5)
rf_grid_search.fit(X_train_scaled, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': [3, 5, 10, 15],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100]})

In [120]:
rf_best_max_depth, rf_best_min_samples_split, rf_best_n_estimators = rf_grid_search.best_params_.values()
rf_best_max_depth, rf_best_min_samples_split, rf_best_n_estimators

(100, 2, 15)

In [121]:
rf_model = RandomForestRegressor(
    n_estimators=rf_best_n_estimators, min_samples_split=rf_best_min_samples_split, max_depth=rf_best_max_depth
)

rf_model.fit(X_train_scaled, y_train);

In [122]:
y_train_pred = rf_model.predict(X_train_scaled)
y_pred = rf_model.predict(X_test_scaled)

In [123]:
print(f"Train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred)}")
print()
print(f"Train MAE = {mean_absolute_error(y_train, y_train_pred)}")
print(f"Test MAE = {mean_absolute_error(y_test, y_pred)}")

Train MSE = 45722076804.311356
Test MSE = 203771154168.69247

Train MAE = 91787.11287523742
Test MAE = 219091.77443745313


In [126]:
# Создадим таблицу с важностями
rf_model_feature_coefs = pd.DataFrame(
    data=rf_model.feature_importances_, index=transformer_names, columns=['Coef']
)


rf_model_feature_coefs.sort_values(by='Coef', ascending=False)

,Coef
scaling__Count_horsepower,0.573549
scaling__Year,0.199813
scaling__Mileage,0.056399
ohe__Engine_type_дизель,0.026487
scaling__Usage_intensity,0.023076
...,...
ohe__Model_Escudo,0.000000
ohe__Model_М-20 Победа,0.000000
ohe__Model_Town Ace Noah,0.000000
ohe__Model_Trax,0.000000


__Вывод:__ Таким образом, нам удалось добиться улучшения модели, и наша ошибка на тестовой и обучающей выборке стала значительно меньше, чем при использовании линейной регресии и случайного леса. 